# Arduino Nano BLE33 Sense

IMU sensor board for telescope ALT/AZ measurements using accelerometer and magnetometer.

**⚠️ Firmware must be uploaded first!** Use `arduino-nano-ble33-sense-upload.ipynb` to upload firmware.

---

## Quick Start (run after every kernel restart)

1. Run **"Setup"** cell
2. Run **"List Serial Ports"** 
3. Run **"Connect to Serial"**
4. Run **"Start Sensor Reader"**
5. Run **"Send STATUS Command"** to verify connection
6. Run **"Display Sensor Values"** to see data

---

## Sensor Output Format

The Arduino sketch outputs 8 tab-separated values:
- `aX, aY, aZ` - Accelerometer (g)
- `mX, mY, mZ` - Magnetometer (µT)
- `temperature` - Temperature (°C)
- `humidity` - Relative Humidity (%RH)

In [1]:
# ============================================
# SETUP
# Run this first after every kernel restart
# ============================================

import serial
import serial.tools.list_ports
import threading
import time
import math

# Auto-detect Arduino Nano 33 BLE port
PORT = None
for port in serial.tools.list_ports.comports():
    # Look for Arduino Nano 33 BLE (VID:PID = 2341:005A or 2341:805A for bootloader)
    if port.vid == 0x2341 and port.pid in (0x005A, 0x805A):
        PORT = port.device
        print(f"✓ Found Nano 33 BLE: {port.device}")
        break
    # Fallback: any Arduino on ACM port
    elif "Arduino" in (port.description or "") or (port.vid == 0x2341):
        PORT = port.device
        print(f"✓ Found Arduino: {port.device} ({port.description})")
        break

if not PORT:
    # Last resort: first ACM port
    acm_ports = [p for p in serial.tools.list_ports.comports() if "ACM" in p.device]
    if acm_ports:
        PORT = acm_ports[0].device
        print(f"⚠️ No Arduino detected, using first ACM port: {PORT}")
    else:
        print("❌ No serial port found! Is Arduino connected?")
        for p in serial.tools.list_ports.comports():
            print(f"     {p.device}: {p.description}")

print(f"✓ Setup complete")
print(f"✓ Port: {PORT}")

✓ Found Nano 33 BLE: /dev/ttyACM0
✓ Setup complete
✓ Port: /dev/ttyACM0


In [2]:
# ============================================
# LIST SERIAL PORTS
# Shows available serial ports
# ============================================

import serial
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

print("Available ports:", [port.device for port in ports])
for port in ports:
    print(f"  {port.device}: {port.description}")

Available ports: ['/dev/ttyS31', '/dev/ttyS30', '/dev/ttyS29', '/dev/ttyS28', '/dev/ttyS27', '/dev/ttyS26', '/dev/ttyS25', '/dev/ttyS24', '/dev/ttyS23', '/dev/ttyS22', '/dev/ttyS21', '/dev/ttyS20', '/dev/ttyS19', '/dev/ttyS18', '/dev/ttyS17', '/dev/ttyS16', '/dev/ttyS15', '/dev/ttyS14', '/dev/ttyS13', '/dev/ttyS12', '/dev/ttyS11', '/dev/ttyS10', '/dev/ttyS9', '/dev/ttyS8', '/dev/ttyS7', '/dev/ttyS6', '/dev/ttyS5', '/dev/ttyS4', '/dev/ttyS3', '/dev/ttyS2', '/dev/ttyS1', '/dev/ttyS0', '/dev/ttyACM1', '/dev/ttyACM0']
  /dev/ttyS31: n/a
  /dev/ttyS30: n/a
  /dev/ttyS29: n/a
  /dev/ttyS28: n/a
  /dev/ttyS27: n/a
  /dev/ttyS26: n/a
  /dev/ttyS25: n/a
  /dev/ttyS24: n/a
  /dev/ttyS23: n/a
  /dev/ttyS22: n/a
  /dev/ttyS21: n/a
  /dev/ttyS20: n/a
  /dev/ttyS19: n/a
  /dev/ttyS18: n/a
  /dev/ttyS17: n/a
  /dev/ttyS16: n/a
  /dev/ttyS15: n/a
  /dev/ttyS14: n/a
  /dev/ttyS13: n/a
  /dev/ttyS12: n/a
  /dev/ttyS11: n/a
  /dev/ttyS10: n/a
  /dev/ttyS9: n/a
  /dev/ttyS8: n/a
  /dev/ttyS7: n/a
  /dev/t

In [3]:
# ============================================
# CONNECT TO SERIAL
# Opens serial connection to Arduino
# ============================================

sensors = serial.Serial(PORT, baudrate=115200, timeout=1)
time.sleep(2)  # Wait for Arduino to finish init
sensors.reset_input_buffer()  # Clear any startup messages
print(f"✓ Connected to {sensors.port}")

✓ Connected to /dev/ttyACM0


In [4]:
# ============================================
# START SENSOR READER
# Background thread that continuously reads sensor data
# ============================================

import threading

accelerometer = {}
magnetometer = {}
environment = {}
_reader_paused = False  # Flag to pause reader during commands

def read_sensors():
    """Read sensor data from Arduino Nano BLE33 Sense via serial."""
    global accelerometer, magnetometer, environment, _reader_paused

    while True:
        if _reader_paused:
            import time
            time.sleep(0.05)  # Sleep while paused
            continue
            
        try:
            str_values = sensors.read_until(expected=b"\r\n").decode().strip().split("\t")
            if len(str_values) == 8:
                accelerometer = {
                    'aX': float(str_values[0]), 
                    'aY': float(str_values[1]), 
                    'aZ': float(str_values[2])
                }
                magnetometer = {
                    'mX': float(str_values[3]), 
                    'mY': float(str_values[4]), 
                    'mZ': float(str_values[5])
                }
                environment = {
                    'temperature': float(str_values[6]),
                    'humidity': float(str_values[7])
                }
            elif len(str_values) == 6:
                accelerometer = {
                    'aX': float(str_values[0]), 
                    'aZ': float(str_values[1]), 
                    'aY': float(str_values[2])
                }
                magnetometer = {
                    'mX': float(str_values[3]), 
                    'mZ': float(str_values[4]), 
                    'mY': float(str_values[5])
                }
        except (ValueError, UnicodeDecodeError):
            pass  # Skip malformed lines

threading.Thread(target=read_sensors, daemon=True).start()
print("✓ Sensor reader thread started")

✓ Sensor reader thread started


## Send Commands to Arduino

Send serial commands to control the sensor. Available commands:
- `RESET` - Reinitialize all sensors
- `STATUS` - Report sensor status and configuration  
- `CALIBRATE` - Run magnetometer calibration (rotate device slowly)
- `STOP` - Pause sensor output
- `START` - Resume sensor output

In [5]:
# ============================================
# SEND COMMAND (helper function)
# Defines send_command() for Arduino communication
# Automatically pauses reader thread during commands
# ============================================

def send_command(cmd: str, wait_response: bool = True, timeout: float = 2.0) -> str:
    """Send a command to the Arduino and optionally wait for response.
    
    Automatically pauses the background reader thread during command execution.
    """
    global _reader_paused
    import time
    
    # Pause reader thread
    _reader_paused = True
    time.sleep(0.1)  # Let reader finish current read
    
    try:
        sensors.reset_input_buffer()
        sensors.write(f"{cmd}\n".encode())
        print(f"Sent: {cmd}")
        
        if not wait_response:
            return ""
        
        response_lines = []
        start_time = time.time()
        
        while (time.time() - start_time) < timeout:
            if sensors.in_waiting:
                line = sensors.readline().decode().strip()
                response_lines.append(line)
                print(line)
                if line.startswith("===") and len(response_lines) > 1:
                    break
                if line.startswith("OK:") or line.startswith("ERROR:"):
                    break
            time.sleep(0.01)
        
        return "\n".join(response_lines)
    finally:
        # Always resume reader thread
        _reader_paused = False

print("✓ send_command() defined (auto-pauses reader thread)")

✓ send_command() defined (auto-pauses reader thread)


In [6]:
# ============================================
# SEND STATUS COMMAND
# Verifies firmware is working - shows sensor status
# ============================================

send_command("STATUS", timeout=3.0)

Sent: STATUS
=== SENSOR STATUS ===
IMU (LSM9DS1): OK
HTS221: OK
Output: ENABLED
Sample Rate: 10 Hz
Mag Offsets (X,Y,Z): 0.00, 0.00, 0.00
--- Current Readings ---
Accel (g): 0.99, -0.01, 0.01
Mag (uT): 48.60, -14.22, -17.09
Temp: 26.2 C
Humidity: 38.4 %RH


'=== SENSOR STATUS ===\nIMU (LSM9DS1): OK\nHTS221: OK\nOutput: ENABLED\nSample Rate: 10 Hz\nMag Offsets (X,Y,Z): 0.00, 0.00, 0.00\n--- Current Readings ---\nAccel (g): 0.99, -0.01, 0.01\nMag (uT): 48.60, -14.22, -17.09\nTemp: 26.2 C\nHumidity: 38.4 %RH\n====================='

In [7]:
# ============================================
# SEND RESET COMMAND
# Reinitializes all sensors
# ============================================

send_command("RESET", timeout=5.0)

Sent: RESET
CMD: Resetting sensors...
INFO: Starting sensor initialization sequence...
INFO: Initializing LSM9DS1 IMU...
DEBUG: Calling IMU.begin()...
DEBUG: IMU.begin() returned OK
INFO: IMU warm-up...
OK: IMU initialized


'CMD: Resetting sensors...\nINFO: Starting sensor initialization sequence...\nINFO: Initializing LSM9DS1 IMU...\nDEBUG: Calling IMU.begin()...\nDEBUG: IMU.begin() returned OK\nINFO: IMU warm-up...\nOK: IMU initialized'

In [8]:
# ============================================
# SEND CALIBRATE COMMAND
# Rotate device slowly in all directions for ~10 seconds
# ============================================

send_command("CALIBRATE", timeout=15.0)

Sent: CALIBRATE
OK: HTS221 initialized


'OK: HTS221 initialized'

In [9]:
# ============================================
# SEND STOP COMMAND
# Pauses sensor output
# ============================================

send_command("STOP")

Sent: STOP
Rotate the sensor slowly in all directions.
.


'Rotate the sensor slowly in all directions.\nCollecting samples...\n.'

In [10]:
# ============================================
# SEND START COMMAND
# Resumes sensor output
# ============================================

send_command("START")

Sent: START
..


'..'

---
## Read Sensor Data

## Coordinate Conversion

Convert between RA/DEC (equatorial) and ALT/AZ (horizontal) coordinate systems.
Requires observer location and current time.

In [11]:
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time
from astropy import units as u
from datetime import datetime, timezone
from typing import NamedTuple

class ObserverLocation(NamedTuple):
    """Observer location on Earth."""
    latitude: float   # degrees, positive North
    longitude: float  # degrees, positive East
    elevation: float  # meters above sea level

# Set your observer location (example: Austin, TX)
OBSERVER_LOCATION = ObserverLocation(
    latitude=30.2672,
    longitude=-97.7431,
    elevation=150.0
)

def get_earth_location(loc: ObserverLocation = OBSERVER_LOCATION) -> EarthLocation:
    """Get astropy EarthLocation from observer location."""
    return EarthLocation(
        lat=loc.latitude * u.deg,
        lon=loc.longitude * u.deg,
        height=loc.elevation * u.m
    )

def radec_to_altaz(
    ra: float, 
    dec: float, 
    obstime: datetime | None = None,
    location: ObserverLocation = OBSERVER_LOCATION
) -> tuple[float, float]:
    """Convert RA/DEC to ALT/AZ for observer location and time.
    
    Args:
        ra: Right Ascension in degrees (0-360) or hours (0-24 if < 24).
        dec: Declination in degrees (-90 to +90).
        obstime: Observation time (UTC). Defaults to now.
        location: Observer location on Earth.
        
    Returns:
        Tuple of (altitude, azimuth) in degrees.
    """
    if ra < 24:
        ra = ra * 15  # Convert hours to degrees
    if obstime is None:
        obstime = datetime.now(timezone.utc)
    
    earth_loc = get_earth_location(location)
    obs_time = Time(obstime)
    altaz_frame = AltAz(obstime=obs_time, location=earth_loc)
    sky_coord = SkyCoord(ra=ra * u.deg, dec=dec * u.deg, frame='icrs')
    altaz_coord = sky_coord.transform_to(altaz_frame)
    
    return altaz_coord.alt.deg, altaz_coord.az.deg

def altaz_to_radec(
    alt: float, 
    az: float, 
    obstime: datetime | None = None,
    location: ObserverLocation = OBSERVER_LOCATION
) -> tuple[float, float]:
    """Convert ALT/AZ to RA/DEC for observer location and time.
    
    Args:
        alt: Altitude in degrees (0 = horizon, 90 = zenith).
        az: Azimuth in degrees (0 = North, 90 = East).
        obstime: Observation time (UTC). Defaults to now.
        location: Observer location on Earth.
        
    Returns:
        Tuple of (ra, dec) in degrees.
    """
    if obstime is None:
        obstime = datetime.now(timezone.utc)
    
    earth_loc = get_earth_location(location)
    obs_time = Time(obstime)
    altaz_frame = AltAz(obstime=obs_time, location=earth_loc)
    altaz_coord = SkyCoord(alt=alt * u.deg, az=az * u.deg, frame=altaz_frame)
    icrs_coord = altaz_coord.transform_to('icrs')
    
    return icrs_coord.ra.deg, icrs_coord.dec.deg

print(f"✓ Observer location: {OBSERVER_LOCATION.latitude:.4f}°N, {OBSERVER_LOCATION.longitude:.4f}°E")
print("✓ Coordinate functions defined: radec_to_altaz(), altaz_to_radec()")

✓ Observer location: 30.2672°N, -97.7431°E
✓ Coordinate functions defined: radec_to_altaz(), altaz_to_radec()


In [17]:
# ============================================
# DISPLAY SENSOR VALUES
# Shows current accelerometer, magnetometer, environment data
# ============================================

print("Accelerometer:", accelerometer)
print("Magnetometer:", magnetometer)
print("Environment:", environment)

Accelerometer: {'aX': 0.99, 'aY': -0.04, 'aZ': 0.01}
Magnetometer: {'mX': -0.26, 'mY': 0.3, 'mZ': -0.81}
Environment: {'temperature': 26.1, 'humidity': 38.74}


In [ ]:
# ============================================
# HOLD ACCELEROMETER SNAPSHOT
# Captures current accelerometer for calculations
# ============================================

hold = accelerometer
print("Snapshot:", hold)

## Calculate the tilt with respect to _x_ (ALT)

From this [post](https://www.digikey.com/en/articles/using-an-accelerometer-for-inclination-sensing).

You need to do this because the accelerometer is sinusoidal in nature.

In [ ]:
import math

print(accelerometer)
tiltX = math.atan(accelerometer['aX'] / math.sqrt(
    (accelerometer['aY'] * accelerometer['aY']) + 
    (accelerometer['aZ'] * accelerometer['aZ'])
)) * (180 / math.pi)
tiltX

### Altitude Calibration

The raw tilt angle from the accelerometer is imperfect due to sensor mounting offset (not perfectly aligned with telescope optical axis). We use a **single-star calibration** to determine the offset:

1. Point telescope at a known star
2. Plate-solve to get true RA/DEC
3. Convert to true ALT/AZ using location + time
4. Offset = `true_alt - sensor_alt`

Since the accelerometer measures gravity (physics), the **scale is inherently 1.0** - only the offset varies due to mounting angle.

```
true_alt = sensor_alt + offset
```

In [ ]:
# Altitude offset calibration (single value from plate-solve)
# Set this after running single-star calibration below
ALT_OFFSET = 0.0  # degrees (will be set by calibrate_altitude())

def get_raw_altitude() -> float:
    """Get raw altitude from accelerometer (before calibration)."""
    ax = accelerometer.get('aX', 0)
    ay = accelerometer.get('aY', 0)
    az = accelerometer.get('aZ', 0)
    return math.atan(ax / math.sqrt(ay*ay + az*az)) * (180 / math.pi)

def get_altitude() -> float:
    """Get calibrated altitude (raw + offset)."""
    return get_raw_altitude() + ALT_OFFSET

def calibrate_altitude(true_alt: float) -> float:
    """Single-star altitude calibration.
    
    Point telescope at a star, plate-solve to get true ALT, then call this.
    
    Args:
        true_alt: True altitude from plate-solve (degrees).
        
    Returns:
        The calculated offset.
        
    Example:
        >>> # After plate-solving: star is at ALT=45.23°
        >>> calibrate_altitude(45.23)
        Offset: 2.15°
        >>> get_altitude()  # Now returns corrected value
    """
    global ALT_OFFSET
    raw = get_raw_altitude()
    ALT_OFFSET = true_alt - raw
    print(f"Raw sensor: {raw:.2f}°")
    print(f"True ALT:   {true_alt:.2f}°")
    print(f"Offset:     {ALT_OFFSET:+.2f}°")
    return ALT_OFFSET

print("✓ Altitude functions defined: get_raw_altitude(), get_altitude(), calibrate_altitude()")
print(f"  Current offset: {ALT_OFFSET:+.2f}°")

### Single-Star Calibration

1. Point at a star, take image, plate-solve → get RA/DEC
2. Run the cell below with your plate-solve result
3. `ALT_OFFSET` is now set for this session

In [ ]:
# Single-star calibration - enter your plate-solve RA/DEC here
# (Run the coordinate conversion cells first to define radec_to_altaz)

PLATE_SOLVE_RA = 83.820   # degrees (e.g., Betelgeuse = 88.79°)
PLATE_SOLVE_DEC = -5.391  # degrees (e.g., Betelgeuse = 7.41°)

# Convert plate-solve RA/DEC to true ALT/AZ
from datetime import datetime
true_alt, true_az = radec_to_altaz(PLATE_SOLVE_RA, PLATE_SOLVE_DEC)
print(f"Plate-solve: RA={PLATE_SOLVE_RA:.3f}°, DEC={PLATE_SOLVE_DEC:.3f}°")
print(f"True ALT/AZ: {true_alt:.2f}°, {true_az:.2f}°")
print()

# Calibrate altitude offset
calibrate_altitude(true_alt)

In [ ]:
# Get current calibrated altitude
print(f"Raw altitude:        {get_raw_altitude():.2f}°")
print(f"Calibrated altitude: {get_altitude():.2f}°")

## Determine Compass Heading (Azimuth)

The magnetometer measures Earth's magnetic field to determine compass heading.

### Magnetometer Calibration Procedure

**⚠️ IMPORTANT: The calibration uses min/max values, not averaging.**

This means you must traverse the **full azimuth range** during the 60-second calibration window:

1. **Position telescope at one azimuth extreme** (e.g., far left limit)
2. **Run the CALIBRATE cell below** - starts 60-second countdown
3. **Slowly rotate through full azimuth range** to the opposite extreme
4. **Timing:** ~180° in 60 sec = 3°/sec (very slow, telescope-safe)
5. **Keep altitude fixed** - don't move up/down during calibration

The algorithm captures the magnetic field envelope (min/max on each axis). Stationary time doesn't hurt - only the extremes matter. But you **must hit both ends** of your azimuth range for good calibration.

Reference: https://www.w3.org/TR/magnetometer/

In [ ]:
def calibrate_heading(expected_heading: float) -> float:
    """Calculate heading offset based on known reference direction.
    
    Args:
        expected_heading: Known compass heading in degrees.
        
    Returns:
        Offset to apply to calculated headings.
    """
    calculated_heading = math.atan2(magnetometer['mY'], magnetometer['mX']) * (180 / math.pi)
    return expected_heading - calculated_heading

# Calibrate with a known heading (e.g., 280 degrees)
offset = calibrate_heading(280)

In [ ]:
# Calculate azimuth (compass heading)
azimuth = int((math.atan2(magnetometer['mZ'], magnetometer['mY']) * (180 / math.pi)) + offset)
azimuth

## Environment Data

Temperature and humidity from the HTS221 sensor.

In [ ]:
print(f"Temperature: {environment.get('temperature', 'N/A')}°C")
print(f"Humidity: {environment.get('humidity', 'N/A')}%RH")

## Sensor Calibration

The `SensorCalibration` class maps raw sensor ALT/AZ to corrected values using plate-solved camera images as ground truth.

**Workflow:**
1. Read sensor ALT/AZ at current position
2. Take image and plate-solve for true RA/DEC
3. Convert RA/DEC to true ALT/AZ
4. Add calibration point (sensor vs true)
5. After multiple points, compute transform
6. Apply transform to future sensor readings

In [ ]:
import numpy as np
from dataclasses import dataclass, field
from typing import Optional
from pathlib import Path
import json

@dataclass
class CalibrationPoint:
    """A single calibration measurement."""
    sensor_alt: float      # Sensor-reported altitude
    sensor_az: float       # Sensor-reported azimuth
    true_alt: float        # True altitude (from plate solve)
    true_az: float         # True azimuth (from plate solve)
    timestamp: datetime    # When measurement was taken
    ra: Optional[float] = None   # Original RA from plate solve
    dec: Optional[float] = None  # Original DEC from plate solve


@dataclass
class SensorTransform:
    """Offset-only transform to correct sensor readings.
    
    Physics provides inherent scale (gravity for ALT, magnetic field for AZ),
    so only mounting offset needs calibration. Scale is always 1.0.
    """
    alt_offset: float = 0.0      # Altitude offset (degrees)
    az_offset: float = 0.0       # Azimuth offset (degrees, includes magnetic declination)
    
    def apply(self, sensor_alt: float, sensor_az: float) -> tuple[float, float]:
        """Apply offset transform to sensor readings."""
        corrected_alt = sensor_alt + self.alt_offset
        corrected_az = (sensor_az + self.az_offset) % 360
        return corrected_alt, corrected_az
    
    def to_dict(self) -> dict:
        """Export transform as dictionary."""
        return {
            'alt_offset': self.alt_offset,
            'az_offset': self.az_offset
        }
    
    @classmethod
    def from_dict(cls, data: dict) -> 'SensorTransform':
        """Create transform from dictionary."""
        # Handle legacy format with scale factors
        return cls(
            alt_offset=data.get('alt_offset', 0.0),
            az_offset=data.get('az_offset', 0.0)
        )


class SensorCalibration:
    """Calibrate IMU sensor against plate-solved camera images.
    
    Uses single-point offset calibration (no scale fitting needed).
    Physics provides inherent scale from gravity/magnetic field.
    
    Calibration is valid until telescope is physically moved.
    Files are saved with timestamps to track calibration age.
    """
    
    def __init__(self, location: ObserverLocation = OBSERVER_LOCATION):
        self.location = location
        self.points: list[CalibrationPoint] = []
        self.transform: Optional[SensorTransform] = None
        self.calibrated_at: Optional[datetime] = None  # When transform was computed
    
    def add_point(
        self, 
        sensor_alt: float, 
        sensor_az: float,
        true_alt: float, 
        true_az: float,
        timestamp: Optional[datetime] = None
    ) -> None:
        """Add a calibration point with known true ALT/AZ."""
        if timestamp is None:
            timestamp = datetime.now(timezone.utc)
        
        point = CalibrationPoint(
            sensor_alt=sensor_alt,
            sensor_az=sensor_az,
            true_alt=true_alt,
            true_az=true_az,
            timestamp=timestamp
        )
        self.points.append(point)
        print(f"Added calibration point {len(self.points)}: "
              f"sensor({sensor_alt:.2f}°, {sensor_az:.2f}°) → "
              f"true({true_alt:.2f}°, {true_az:.2f}°)")
    
    def add_point_from_radec(
        self,
        sensor_alt: float,
        sensor_az: float,
        ra: float,
        dec: float,
        timestamp: Optional[datetime] = None
    ) -> None:
        """Add a calibration point from plate-solved RA/DEC."""
        if timestamp is None:
            timestamp = datetime.now(timezone.utc)
        
        # Convert RA/DEC to ALT/AZ
        true_alt, true_az = radec_to_altaz(ra, dec, timestamp, self.location)
        
        point = CalibrationPoint(
            sensor_alt=sensor_alt,
            sensor_az=sensor_az,
            true_alt=true_alt,
            true_az=true_az,
            timestamp=timestamp,
            ra=ra,
            dec=dec
        )
        self.points.append(point)
        print(f"Added calibration point {len(self.points)}: "
              f"sensor({sensor_alt:.2f}°, {sensor_az:.2f}°) → "
              f"RA/DEC({ra:.4f}°, {dec:.4f}°) → "
              f"true ALT/AZ({true_alt:.2f}°, {true_az:.2f}°)")
    
    def compute_transform(self) -> SensorTransform:
        """Compute offset transform from calibration points.
        
        Only one point needed - physics provides inherent scale.
        Multiple points will average the offsets for better accuracy.
        """
        if len(self.points) < 1:
            raise ValueError("Need at least 1 calibration point")
        
        # Compute offset as mean difference (true - sensor)
        alt_offsets = [p.true_alt - p.sensor_alt for p in self.points]
        
        # Handle azimuth wrap-around
        az_offsets = []
        for p in self.points:
            diff = p.true_az - p.sensor_az
            diff = (diff + 180) % 360 - 180  # Normalize to [-180, 180]
            az_offsets.append(diff)
        
        alt_offset = np.mean(alt_offsets)
        az_offset = np.mean(az_offsets)
        
        self.transform = SensorTransform(
            alt_offset=alt_offset,
            az_offset=az_offset
        )
        self.calibrated_at = datetime.now(timezone.utc)
        
        n = len(self.points)
        print(f"Transform computed from {n} point{'s' if n > 1 else ''}:")
        print(f"  ALT: corrected = sensor + {alt_offset:+.2f}°")
        print(f"  AZ:  corrected = sensor + {az_offset:+.2f}°")
        print(f"  Calibrated at: {self.calibrated_at.strftime('%Y-%m-%d %H:%M:%S')} UTC")
        
        # Show residuals if multiple points
        if n > 1:
            sensor_alt = np.array([p.sensor_alt for p in self.points])
            sensor_az = np.array([p.sensor_az for p in self.points])
            true_alt = np.array([p.true_alt for p in self.points])
            true_az = np.array([p.true_az for p in self.points])
            
            pred_alt = sensor_alt + alt_offset
            pred_az = (sensor_az + az_offset) % 360
            print(f"  Residuals: ALT σ={np.std(true_alt - pred_alt):.3f}°, "
                  f"AZ σ={np.std((true_az - pred_az + 180) % 360 - 180):.3f}°")
        
        return self.transform
    
    def apply(self, sensor_alt: float, sensor_az: float) -> tuple[float, float]:
        """Apply transform to get corrected ALT/AZ."""
        if self.transform is None:
            raise ValueError("Transform not computed. Call compute_transform() first.")
        return self.transform.apply(sensor_alt, sensor_az)
    
    def get_current_sensor_altaz(self) -> tuple[float, float]:
        """Get current ALT/AZ from sensor."""
        ax = accelerometer.get('aX', 0)
        ay = accelerometer.get('aY', 0)
        az_accel = accelerometer.get('aZ', 0)
        
        sensor_alt = math.atan(ax / math.sqrt(ay*ay + az_accel*az_accel)) * (180 / math.pi)
        
        mx = magnetometer.get('mX', 0)
        my = magnetometer.get('mY', 0)
        
        sensor_az = math.atan2(my, mx) * (180 / math.pi)
        if sensor_az < 0:
            sensor_az += 360
        
        return sensor_alt, sensor_az
    
    def save(self, directory: str = "../data") -> str:
        """Save calibration to timestamped JSON file.
        
        Args:
            directory: Directory to save calibration file.
            
        Returns:
            Path to saved file.
        """
        Path(directory).mkdir(parents=True, exist_ok=True)
        
        # Generate timestamped filename
        ts = self.calibrated_at or datetime.now(timezone.utc)
        filename = f"sensor_calibration_{ts.strftime('%Y%m%d_%H%M%S')}.json"
        filepath = Path(directory) / filename
        
        data = {
            'calibrated_at': ts.isoformat(),
            'location': {
                'latitude': self.location.latitude,
                'longitude': self.location.longitude,
                'elevation': self.location.elevation
            },
            'transform': self.transform.to_dict() if self.transform else None,
            'points': [
                {
                    'sensor_alt': p.sensor_alt,
                    'sensor_az': p.sensor_az,
                    'true_alt': p.true_alt,
                    'true_az': p.true_az,
                    'timestamp': p.timestamp.isoformat(),
                    'ra': p.ra,
                    'dec': p.dec
                }
                for p in self.points
            ]
        }
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✓ Calibration saved: {filepath}")
        return str(filepath)
    
    @classmethod
    def load(cls, filepath: str) -> 'SensorCalibration':
        """Load calibration from JSON file.
        
        Shows warning if calibration is old (>24 hours).
        """
        with open(filepath, 'r') as f:
            data = json.load(f)
        
        loc = ObserverLocation(**data['location'])
        cal = cls(location=loc)
        
        # Load calibration timestamp
        if 'calibrated_at' in data:
            cal.calibrated_at = datetime.fromisoformat(data['calibrated_at'])
        
        for p in data['points']:
            point = CalibrationPoint(
                sensor_alt=p['sensor_alt'],
                sensor_az=p['sensor_az'],
                true_alt=p['true_alt'],
                true_az=p['true_az'],
                timestamp=datetime.fromisoformat(p['timestamp']),
                ra=p.get('ra'),
                dec=p.get('dec')
            )
            cal.points.append(point)
        
        if data.get('transform'):
            cal.transform = SensorTransform.from_dict(data['transform'])
        
        # Calculate age and warn if old
        print(f"✓ Loaded calibration with {len(cal.points)} points from {filepath}")
        if cal.calibrated_at:
            age = datetime.now(timezone.utc) - cal.calibrated_at
            age_hours = age.total_seconds() / 3600
            age_str = f"{age_hours:.1f} hours" if age_hours < 48 else f"{age_hours/24:.1f} days"
            
            if age_hours > 24:
                print(f"⚠️  WARNING: Calibration is {age_str} old!")
                print(f"   If telescope has moved, recalibrate with a fresh plate-solve.")
            else:
                print(f"  Calibrated {age_str} ago ({cal.calibrated_at.strftime('%Y-%m-%d %H:%M')} UTC)")
        
        return cal
    
    @staticmethod
    def list_calibrations(directory: str = "../data") -> list[str]:
        """List available calibration files, newest first."""
        path = Path(directory)
        if not path.exists():
            return []
        files = sorted(path.glob("sensor_calibration_*.json"), reverse=True)
        return [str(f) for f in files]


# Create calibration instance
calibration = SensorCalibration(OBSERVER_LOCATION)
print(f"✓ SensorCalibration initialized for {OBSERVER_LOCATION.latitude:.4f}°N")
print("  Single-point calibration enabled (offset-only, no scale fitting)")

### Calibration Workflow

Run through these cells to calibrate the sensor against plate-solved images.

In [ ]:
# Step 1: Get current sensor reading
sensor_alt, sensor_az = calibration.get_current_sensor_altaz()
print(f"Current sensor reading: ALT={sensor_alt:.2f}°, AZ={sensor_az:.2f}°")

In [ ]:
# Step 2: After plate-solving image, add calibration point
# Replace these values with your plate-solve results

# Example: M42 Orion Nebula plate solve result
plate_solve_ra = 83.820  # degrees (or hours if < 24)
plate_solve_dec = -5.391  # degrees

# Add the calibration point (uses current sensor reading and plate solve result)
calibration.add_point_from_radec(
    sensor_alt=sensor_alt,
    sensor_az=sensor_az,
    ra=plate_solve_ra,
    dec=plate_solve_dec
)

In [ ]:
# Step 3: (Optional) Add more points for better accuracy
# More points = averaged offset, reduces measurement noise

# View current calibration points
print(f"Calibration points: {len(calibration.points)}")
for i, p in enumerate(calibration.points, 1):
    print(f"  {i}: sensor({p.sensor_alt:.1f}°, {p.sensor_az:.1f}°) → true({p.true_alt:.1f}°, {p.true_az:.1f}°)")

In [ ]:
# Step 4: Compute the transform (only 1 point needed!)
if len(calibration.points) >= 1:
    calibration.compute_transform()
else:
    print("Need at least 1 calibration point. Run Steps 1-2 first.")

In [ ]:
# Step 5: Apply transform to get corrected position
sensor_alt, sensor_az = calibration.get_current_sensor_altaz()

if calibration.transform:
    corrected_alt, corrected_az = calibration.apply(sensor_alt, sensor_az)
    print(f"Raw sensor:  ALT={sensor_alt:.2f}°, AZ={sensor_az:.2f}°")
    print(f"Corrected:   ALT={corrected_alt:.2f}°, AZ={corrected_az:.2f}°")
    
    # Convert to RA/DEC for display
    ra, dec = altaz_to_radec(corrected_alt, corrected_az)
    print(f"Estimated:   RA={ra:.4f}° ({ra/15:.4f}h), DEC={dec:.4f}°")
else:
    print("Transform not computed yet. Run Step 4 first.")

In [ ]:
# Save/Load calibration for later sessions
# Files are timestamped: sensor_calibration_YYYYMMDD_HHMMSS.json

CALIBRATION_DIR = "data"

# List available calibrations (newest first)
print("Available calibrations:")
for f in SensorCalibration.list_calibrations(CALIBRATION_DIR):
    print(f"  {f}")

# Save current calibration (creates timestamped file)
calibration.save(CALIBRATION_DIR)

# Load most recent calibration
files = SensorCalibration.list_calibrations(CALIBRATION_DIR)
if files:
    calibration = SensorCalibration.load(files[0])